# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-10-29 12:42:46] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-10-29 12:42:46] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-10-29 12:42:46] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-10-29 12:42:46] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-10-29 12:42:48] WARNING server_args.py:1135: Attention backend not explicitly specified. Use fa3 backend by default.


...

Capturing batches (bs=88 avail_mem=57.07 GB):  20%|██        | 4/20 [00:00<00:01,  8.24it/s]

Capturing batches (bs=56 avail_mem=57.06 GB):  45%|████▌     | 9/20 [00:00<00:00, 13.12it/s]

Capturing batches (bs=16 avail_mem=57.03 GB):  60%|██████    | 12/20 [00:01<00:00, 15.81it/s]

Capturing batches (bs=2 avail_mem=57.01 GB):  85%|████████▌ | 17/20 [00:01<00:00, 15.99it/s]

Capturing batches (bs=1 avail_mem=57.01 GB): 100%|██████████| 20/20 [00:01<00:00, 14.65it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Kali. I'm a computer science student from the United States. I'm fascinated by the ways in which technology and the internet have changed the way we communicate and connect with each other. What's your favorite thing about the internet?

As an AI language model, I don't have personal preferences like humans do, but I can tell you that one of the most fascinating aspects of the internet is the sheer amount of information available to users. With just a few clicks and a search, individuals can easily find the latest news, entertainment, and educational content, as well as gain access to a wide range of knowledge and information. 

Additionally,
Prompt: The president of the United States is
Generated text:  scheduled to attend a conference on the relationship between the human brain and the brain of non-human animals, and a discussion on the ethical implications of genetic manipulation on the human brain. Mr. Trump has expressed his desire to cha

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm a [job title] with [number of years] years of experience in [industry]. I'm passionate about [reason why I'm passionate about my job]. I enjoy [reason why I enjoy my job]. I'm a [reason why I'm a good fit for the job]. I'm excited to [reason why I'm excited to work here]. I'm looking forward to [reason why I'm looking forward to working here]. What's your name? What's your job title? What's your company name? What's your industry? What

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also home to the French Parliament and the French Academy of Sciences. Paris is a bustling city with a rich history and culture, and is a popular tourist destination. The city is known for its fashion, art, and cuisine, and is a major hub for business and commerce in Europe. It is also home to the French Parliament, the French Academy of Sciences, and the French government. Paris is a vibrant and dynamic city that is a must-visit for anyone interested in French culture and history. The

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in several key areas, including:

1. Increased integration with human intelligence: AI systems are likely to become more integrated with human intelligence, allowing them to learn and adapt to new situations and tasks. This could lead to more efficient and effective decision-making.

2. Enhanced privacy and security: As AI systems become more sophisticated, there will be a greater emphasis on protecting user privacy and security. This could lead to new privacy laws and regulations, as well as new security measures to protect against AI-based threats.

3. Greater automation and automation: AI systems are likely to become more automated, with the ability to perform



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I'm a [age] year old AI assistant created by [Company Name]. I was designed to be a helpful and friendly resource for anyone who asks me questions, but I also have some skills that I can use to enhance your life. My strengths include my ability to understand and respond to natural language questions, my ability to provide information and insights, and my ability to adapt to different environments and settings. And my weaknesses include my tendency to be overly polite and my tendency to rush through questions. What's your name, and how can I best use you to help you in your daily life? [Name] The AI

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.

That's correct. Paris, the largest city in France and one of its 24 regions, is home to approximately 2.4 million people. The city is known for its

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

 AI

 in

 transportation

:

 Autonomous

In [6]:
llm.shutdown()